In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision.transforms import ToTensor
from torchvision import datasets
from tqdm import tqdm

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
training_data.data.shape

torch.Size([60000, 28, 28])

In [3]:
batch_size = 8
training_loader =DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle= False)


In [4]:
img, label = next(iter(training_loader))

In [5]:
img.shape, label

(torch.Size([8, 1, 28, 28]), tensor([2, 6, 4, 7, 3, 4, 2, 1]))

In [7]:
img.shape[0]

4

In [6]:
class Custom_CNN(nn.Module):
    """Custom CNN class for image classification"""
    def __init__(self, n_classes = 10):
        """constructor for initialisation"""
        super().__init__()
        """initialise parents method first so we can use methods from parent class without overriding"""
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1), # [B, 16, 28, 28]
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 32,kernel_size=3, padding=1), # [B, 32, 28, 28]
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size= 3 , padding=1), #[B, 64, 28, 28]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), #[B, 64, 14, 14]
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1), # [B, 128, 14, 14]
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), # [B, 128, 14, 14]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), #[B, 128, 7, 7]
            nn.AdaptiveAvgPool2d((1, 1)), # [B, 128]
            
        )

        self.linear_layer = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.conv_layer(x)
        # print(x.shape)
        x = x.view((x.shape[0], -1))
        # print(x.shape)
        x = self.linear_layer(x)
        return x
    

In [44]:
img, label = next(iter(training_loader))

In [8]:
model = Custom_CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
y_pred.shape

NameError: name 'y_pred' is not defined

In [10]:
for epoch in range(10):
    running_loss = 0
    for img, label in tqdm(training_loader):
        # forward pass, comput output tensors from input tensors
        y_pred = model(img)

        # compute loss
        loss = criterion(y_pred, label)

        # zero gradient
        optimizer.zero_grad()

        # back propagation to compute gradients with respect to loss
        loss.backward()

        # update parameter
        optimizer.step()
            
        running_loss+= loss.item()
    print(f"epoch: {epoch + 1}, average loss: {running_loss/len(training_loader)}")

    correct, total = 0, 0
    for img, label in test_loader:
        outputs = model(img)
        y_pred = outputs.argmax(dim = 1)
        correct += (y_pred == label).sum().item()
        total += len(y_pred)

    print(f"evaluation accuracy: {correct / total}")

        
        

        

100%|██████████| 7500/7500 [02:46<00:00, 45.03it/s]


epoch: 1, average loss: 2.302831979115804
evaluation accuracy: 0.1058


100%|██████████| 7500/7500 [02:46<00:00, 45.16it/s]


epoch: 2, average loss: 2.301485323047638
evaluation accuracy: 0.1629


100%|██████████| 7500/7500 [02:45<00:00, 45.34it/s]


...

epoch: 8, average loss: 0.6955820003921787
evaluation accuracy: 0.7575


100%|██████████| 7500/7500 [02:47<00:00, 44.80it/s]


epoch: 9, average loss: 0.6438950349072615
evaluation accuracy: 0.7774


100%|██████████| 7500/7500 [02:50<00:00, 44.07it/s]


epoch: 10, average loss: 0.6041845050953328
evaluation accuracy: 0.7903


In [117]:
len(y_pred)


4

In [109]:
y_pred.argmax(1)

tensor([1, 1, 1, 1])

In [110]:
label

tensor([9, 5, 4, 0])

In [111]:
y_pred.argmax(1) == label

tensor([False, False, False, False])

In [107]:
y_pred.shape

torch.Size([4, 10])

In [112]:
(y_pred.argmax(1) == label).sum()

tensor(0)

In [89]:
running_loss

2.3402626514434814